In [1]:
import pandas as pd
import numpy as np
import re
from functools import partial, reduce
import warnings
warnings.filterwarnings("ignore")

In [2]:
year = '2017'
data_out = '../data/processed_data/'
data_in = '../data/data_' + year + '/'

## Preprocess numeric

In [3]:
column_info = pd.read_csv(data_out +'plan_summary.csv')
plan_data = pd.read_csv(data_in + 'Plan_Attributes_PUF_' + year + '.csv',encoding='cp1252')

act_value = plan_data[['IssuerActuarialValue','AVCalculatorOutputNumber']].copy()
act_value =  act_value.fillna(0)
act_value['IssuerActuarialValue'] = act_value['IssuerActuarialValue'].apply( lambda x: float(str(x).replace('%','')) )
act_value['AVCalculatorOutputNumber'] = act_value['AVCalculatorOutputNumber']*100
act_value['act_value'] = act_value.max(axis=1)

plan_data['act_value'] = act_value['act_value']

good_columns = column_info['Column_Name'][ (column_info['Missing_Values'] <= 3000) & (column_info['Missing_Values'] >= 0) & (column_info['Unique_Values'] >= 90) 
                  & (column_info['Unique_Values'] <= 370) ]
good_columns = list(good_columns) + ['act_value']

#fix the set of columns to remove bad ones
bad_columns = ['SBCHavingSimplefractureCoinsurance', 
 'SBCHavingSimplefractureCopayment', 'SBCHavingSimplefractureDeductible', 'ServiceAreaID']
good_columns = list(set(good_columns) - set(bad_columns))


# https://stackoverflow.com/questions/4703390/how-to-extract-a-floating-number-from-a-string
clean_columns = plan_data[['ServiceAreaId','IssuerId']].copy()

for column in good_columns:
    #try to extract the text
    test_column = plan_data[column].copy()
    test_column = test_column.fillna(0)
    
    #if its just an int go with that
    int_column = test_column.apply(lambda x : float( (re.findall(r'^-?\d+(?:\.\d+)$',str(x)) +['0']) [0]) )
    if int_column.nunique() >= 3:
        clean_columns[column] = int_column
        
    #otherwise try something else
    dollar_column = test_column.apply(lambda x : int((re.findall('\\$(\\d+)',str(x) ) +['0'])[0]) )
    if dollar_column.nunique() >= 3:
        clean_columns[column] = dollar_column

print(clean_columns.columns)

#IssuerActuarialValue, AVCalculatorOutputNumber

clean_columns.describe()
clean_columns.to_csv(data_out  + 'issuer_numeric_' +year +'.csv')

Index(['ServiceAreaId', 'IssuerId', 'TEHBOutOfNetFamilyPerPersonMOOP',
       'TEHBOutOfNetFamilyPerGroupMOOP', 'SBCHavingDiabetesCoinsurance',
       'SBCHavingaBabyCopayment', 'EHBPercentTotalPremium',
       'TEHBInnTier1IndividualMOOP', 'SBCHavingDiabetesCopayment', 'act_value',
       'TEHBInnTier1FamilyPerGroupMOOP', 'TEHBOutOfNetIndividualMOOP',
       'SBCHavingDiabetesDeductible', 'TEHBCombInnOonFamilyPerGroupMOOP',
       'TEHBInnTier1FamilyPerPersonMOOP', 'SBCHavingaBabyCoinsurance',
       'SBCHavingaBabyDeductible', 'OutOfServiceAreaCoverageDescription',
       'TEHBCombInnOonIndividualMOOP', 'TEHBCombInnOonFamilyPerPersonMOOP'],
      dtype='object')


## Issuer characteristics

In [4]:
plan_df = pd.read_csv(data_in + 'Plan_Attributes_PUF_' + year + '.csv',encoding='cp1252')
plan_summary = pd.read_csv( data_out + 'plan_summary.csv', index_col=None)


#determine which values are missing
no_missing_values = plan_summary[(plan_summary['Missing_Values'] == 0) & 
                                 (plan_summary['Unique_Values'] <= 34)]['Column_Name'].to_list()
no_missing_values.extend(['IssuerId','ServiceAreaId','StandardComponentId','StateCode'])
no_missing_values.remove('PlanEffectiveDate')
no_missing_values.remove('DesignType')


# Treat columns as continous
cleaned_plan_df = plan_df[no_missing_values]
continuous = ['FirstTierUtilization','BeginPrimaryCareCostSharingAfterNumberOfVisits',
              'BeginPrimaryCareDeductibleCoinsuranceAfterNumberOfCopays']
cleaned_plan_df['FirstTierUtilization'] = cleaned_plan_df['FirstTierUtilization'].str.replace('%','')
cleaned_plan_df[continuous] = cleaned_plan_df[continuous].astype(str).astype(float) 

# Get CSRVariationType binary
def largest_dummies(cleaned_plan_df, col_name,number_dummies):
    categories = cleaned_plan_df[col_name].value_counts()
    largest = categories.head(number_dummies).index.to_list()
    rest = categories[number_dummies:].to_list()
    cleaned_plan_df[col_name + 'Other'] = 1
    for j in range(len(largest)):
        category_name = largest[j]
        cleaned_plan_df.loc[ cleaned_plan_df[col_name] == largest[j], col_name+str(category_name)] = 1
        cleaned_plan_df.loc[ cleaned_plan_df[col_name] != largest[j], col_name+str(category_name)] = 0
        cleaned_plan_df.loc[ cleaned_plan_df[col_name] == largest[j], col_name + 'Other'] = 0
    return cleaned_plan_df

cleaned_plan_df = largest_dummies(cleaned_plan_df, 'CSRVariationType',4)
cleaned_plan_df = largest_dummies(cleaned_plan_df, 'IssuerId',10)
cleaned_plan_df = largest_dummies(cleaned_plan_df, 'StateCode',50)

dummy_cols = ['BusinessYear',
 'SourceName',
 'MarketCoverage',
 'DentalOnlyPlan',
 'IsNewPlan',
 'PlanType',
 'MetalLevel',
 'QHPNonQHPTypeId',
 'CompositeRatingOffered',
 'ChildOnlyOffering',
 'OutOfCountryCoverage',
 'OutOfServiceAreaCoverage',
 'NationalNetwork',
 'MultipleInNetworkTiers',
 'InpatientCopaymentMaximumDays']
cleaned_plan_df = pd.get_dummies(cleaned_plan_df, columns = dummy_cols)
cleaned_plan_df

issuer_numeric = pd.read_csv(data_out + 'issuer_numeric_' + year + '.csv')
issuer_numeric = issuer_numeric.iloc[: , 1:]
merged_df = cleaned_plan_df.merge(issuer_numeric, left_index=True,right_index=True,suffixes=('', '_y'))

#clean up the columns 
relevant_cols = list(merged_df.columns)
for word in ['IssuerId','ServiceAreaId','IssuerId_y','ServiceAreaId_y']:
    relevant_cols.remove(word)
relevant_cols = ['IssuerId','ServiceAreaId','StandardComponentId'] + relevant_cols 
merged_df = merged_df[relevant_cols]
merged_df

#drop the dental plan data?
print(merged_df.shape)
merged_df  = merged_df[plan_df['DentalOnlyPlan']=='No']
print(merged_df.shape)

merged_df.to_csv(data_out + 'issuer_characteristics_'+ year + '.csv', index=False)


## Adding the HIX data
#add in the hix data
cms_data = pd.read_csv( data_out + 'issuer_characteristics_'+ year + '.csv')
#clean up planids in the HIX data...
hix_data = pd.read_csv(data_in + 'plans_'+ year + '.csv')
hix_data['PLANID2'] = hix_data['PLANID'] #+ new_plan_df['AREA'].apply(lambda x : '-' + x[-2:])
hix_data['PLANID2'] = hix_data['PLANID2'].apply(lambda x : x[:-3])
no_dash = hix_data['PLANID'].apply(lambda x : x.find('-') == -1 )
hix_data['PLANID2'][no_dash] = hix_data['PLANID'][no_dash]

hix_data = hix_data[['PLANID2','PREMIC', 'PREMI27', 'PREMI50', 'PREMI2C30', 'PREMC2C30']]
hix_data = hix_data.fillna(0)

#take the median for a plan... unclear what area means...
hix_data = hix_data.groupby('PLANID2',as_index=False).median()

print(hix_data['PLANID2'].nunique())
print(hix_data['PREMI50'].nunique())

#first merge on ones where we know the service area?
merged_df = cms_data.merge(hix_data,how='left', left_on=['StandardComponentId'], 
                              right_on=['PLANID2'])
merged_df = merged_df.fillna(0)

print(merged_df['PLANID2'].nunique())
print(merged_df['PREMI50'].nunique())

merged_df = merged_df.drop(labels=['PLANID2'],axis=1)
merged_df.to_csv(data_out + 'issuer_characteristics_' + year + '.csv', index=False)

(21365, 129)
(18556, 129)
7269
6305
3126
3008


## County Characteristics

In [5]:
acs1 = pd.read_csv(data_in + 'ACSDP5Y' + year + '.DP03_data_with_overlays.csv', na_values=['(X)']) #, encoding='cp1252'
acs2 = pd.read_csv(data_in + 'ACSDP5Y' + year + '.DP05_data_with_overlays.csv', na_values=['(X)']) 
acs3 = pd.read_csv(data_in + 'ACSST5Y' + year + '.S1701_data_with_overlays.csv', na_values=['(X)'])

#DP03_0095E - health care coverage

def remove_cols(df):
    
    #save county
    county = df['GEO_ID']
    county = county.str[-5:]
    county = county.astype(str) 
    county = county.loc[1:]
    
    #find the bad columns
    cols = list(df.columns)
    trim_cols1 = []
    for col in cols:
        income_col = 'C01_' in col and 'E' in col
        if 'PE' in col or 'DP03_0095E' in col or income_col:
            trim_cols1.append(col)

    df = df[trim_cols1]
    df = df.loc[1:]
    df.fillna(0, inplace = True)
    
    
    #filter out bad columns
    df['County'] = county
    
    return df


dfs = []

for i in [acs1,acs2,acs3]:
    dfs.append(remove_cols(i))

for i in dfs:
    print(i.shape)


[acs1, acs2,acs3] = dfs
merge = partial(pd.merge, on=['County'], how='outer')
all_acs = reduce(merge, dfs)
all_acs.to_csv(data_out + 'county_characteristics_' + year + '.csv', index=False)


# Comment this section of the code for 2017 data - START
if year == '2016':
    csr = pd.read_csv(data_in + 'csrzipcounty2016.csv', na_values=['.'])
    csr.columns = csr.iloc[2]
    csr = csr.iloc[4:-1]
    csr.reset_index(drop=True)
    csr

    csr_clean = csr.copy()
    csr_clean = csr_clean
    for column in list(csr.columns)[3:]:
        csr_clean[column] = csr_clean[column].fillna('0')
        csr_clean[column] = csr_clean[column].apply(lambda x: float((str(x)).replace('$','').replace(',','')) )
        
    merged_acs = all_acs.merge(csr_clean, how='left', left_on='County', right_on='FIPS County Code')
    merged_acs.fillna(0, inplace = True)
    merged_acs.to_csv(data_out + 'county_characteristics_' + year + '.csv', index=False)


(3220, 139)
(3220, 90)
(3220, 62)


## Merge characteristics

In [6]:
enroll_df = pd.read_csv(data_in + year + '-Issuer-Enrollment-Disenrollment-Report.csv',na_values=['*'])
enroll_df = enroll_df[['HIOS ID','Policy County FIPS Code','Ever Enrolled Count']].fillna('0')
#clean the column
enroll_df['Ever Enrolled Count'] = enroll_df['Ever Enrolled Count'].apply(lambda x: float(str(x).replace(',','')) )

service_df = pd.read_csv(data_in + 'ServiceArea_PUF_'  + year + '.csv', encoding='cp1252')
service_df = service_df[['County','ServiceAreaId','IssuerId']]
service_df = service_df.drop_duplicates()
issuer_df = pd.read_csv(data_out + 'issuer_characteristics_' +year +'.csv')

issuer_service = issuer_df.merge(service_df, how='inner', on=['ServiceAreaId', 'IssuerId'])
issuer_service = issuer_service[ ~issuer_service['County'].isna() ]

#reorder the columns, drop service area
col_order= ['IssuerId','County'] + list(issuer_service.columns)[2:-1]
issuer_service = issuer_service[ col_order ]

#group by county
#pre_cols = list(issuer_service.columns)
issuer_service = issuer_service.groupby(['IssuerId','County','StandardComponentId','StateCode'],as_index=False).median()
issuer_service_count = issuer_service.groupby(['IssuerId','County','StateCode'],as_index=False)['StandardComponentId'].count()
issuer_service = issuer_service.groupby(['IssuerId','County','StateCode'],as_index=False).mean()
issuer_service['Plan Counts'] = issuer_service_count['StandardComponentId']
issuer_service = issuer_service.rename(columns={'StateCode':'State'})
#post_cols = list(issuer_service.columns)
print(issuer_service.shape)
#print(len(pre_cols),len(enroll_df))

#fix issue with merging on float
enroll_df['Policy County FIPS Code'] = enroll_df['Policy County FIPS Code'].apply( lambda x : (int(x)) )
enroll_df['HIOS ID'] = enroll_df['HIOS ID'].apply( lambda x : (int(x)) )
issuer_service['County'] = issuer_service['County'].apply( lambda x : (int(x)) )
issuer_service['IssuerId'] = issuer_service['IssuerId'].apply( lambda x : (int(x)) )

enroll_issuer = enroll_df.merge(issuer_service, how='left', right_on=['County', 'IssuerId'],
                                     left_on=['Policy County FIPS Code','HIOS ID'])
# enroll_issuer.to_csv('test3.csv')
enroll_issuer['County'] = enroll_issuer['Policy County FIPS Code']
enroll_issuer['IssuerId'] = enroll_issuer['HIOS ID']
enroll_issuer = enroll_issuer.fillna(0)
print(enroll_issuer.shape)


county = pd.read_csv(data_out + 'county_characteristics_' + year + '.csv')
result = enroll_issuer.merge(county, how='left', left_on='County', right_on='County')
result.fillna(0,inplace=True)
result = result[result['DP03_0095E'] > 0] # only plans with enrollees
result =result[~result['Policy County FIPS Code'].isna()]


#weird thing - need to remove state?
if year == '2016':
    result = result.rename(columns={'State_x':'State'})

# process columns
all_cols  = list(result.columns)
keys = ['HIOS ID','Policy County FIPS Code','IssuerId','County','State']
delete_key = ['State_y', 'FIPS County Code', 'County Name']

for key in keys:
    all_cols.remove(key)

if year == '2016':
    for key in delete_key: 
        all_cols.remove(key)
    
#delete bad columns/clean up census data
all_cols2 = []

for col in all_cols:
    #fix cols from census data
    result[col] = result[col].apply(lambda x : float(str(x).replace('-','0').replace('N','0')))
    
    #clean up cols with no variance  
    if result[col].std() > 0:
        all_cols2.append(col)  

result[keys + all_cols2].to_csv(data_out + 'merged_characteristics_' + year + '.csv',index=False)

(4537, 132)
(5698, 135)


In [7]:
if year != '2016':
    merged_characteristics_2017 = pd.read_csv(data_out + 'merged_characteristics_' + year + '.csv')
    list2 = list(merged_characteristics_2017.columns)
    #use 2016 as a benchmark
    merged_characteristics = pd.read_csv(data_out + 'merged_characteristics_2016.csv')
    list1 = list(merged_characteristics.columns)
    list_missing = list(set(list1).difference(list2))
    print("Missing values in first list:", list_missing)

    for col in list_missing:
        merged_characteristics_2017[col] = 0
        
    merged_characteristics_2017.to_csv(data_out + 'merged_characteristics_' + year + '.csv',index=False)

Missing values in first list: ['DP05_0032PE', 'DP05_0004PE', 'OutOfCountryCoverage_YES', 'IssuerId56503', 'Average Monthly Advanced CSR Payment for Consumers with 87%', 'ChildOnlyOffering_Allows Child-Only', 'OutOfServiceAreaCoverage_YES', 'Number of Consumers with CSR (AV of 73%/87%/94%)', 'Number of Consumers with CSR AV of 87%', 'DP05_0018PE', 'IssuerId84670', 'BusinessYear_2016', 'Number of Consumers with CSR AV of 94%', 'DP05_0028PE', 'IssuerId14002', 'Average Monthly Advanced CSR Payment for Consumers with 94%', 'StateCodeSC', 'Average Monthly Advanced CSR Payment for Consumers with 73%', 'StateCodeAL', 'Number of Consumers with CSR AV of 73%', 'InpatientCopaymentMaximumDays_1', 'PlanType_Indemnity', 'IssuerId68781', 'PREMIC', 'IssuerId27357', 'NationalNetwork_NO', 'Total Number of Consumers', 'InpatientCopaymentMaximumDays_2']
